# Tuning

In [1]:
import sys
sys.path.append('/Users/c242587/Desktop/projects/git/ngboost')

In [17]:
from ngboost import NGBClassifier, NGBRegressor
from ngboost.distns import k_categorical, Normal
from ngboost.scores import LogScore

from sklearn.datasets import load_breast_cancer, load_boston
from sklearn.model_selection import train_test_split

X, Y = load_boston(True)
X_reg_train, X_reg_test, Y_reg_train, Y_reg_test = train_test_split(X, Y, test_size=0.2)

X, y = load_breast_cancer(True)
y[0:15] = 2 # artificially make this a 3-class problem instead of a 2-class problem
X_cls_train, X_cls_test, Y_cls_train, Y_cls_test  = train_test_split(X, y, test_size=0.2)

## Staged Prediction

All fit NGBoost objects support staged prediction.

In [8]:
ngb_cls = NGBClassifier(Dist=k_categorical(3), Score=LogScore, n_estimators=500, verbose=False).fit(X_cls_train, Y_cls_train)

For instance, to get the predictions on the first 5 examples after fitting 415 base learners, use:

In [9]:
preds = ngb_cls.staged_predict(X_cls_test)
preds[415][0:5]

array([1, 1, 1, 1, 1])

In [10]:
pred_dists = ngb_cls.staged_pred_dist(X_cls_test)
pred_dists[415][0:5].params

{'p0': array([0.0036683 , 0.00370816, 0.00366828, 0.00366828, 0.00366828]),
 'p1': array([0.99608409, 0.99619704, 0.99607732, 0.99607732, 0.99607732]),
 'p2': array([2.47605121e-04, 9.47955927e-05, 2.54405628e-04, 2.54405628e-04,
        2.54405628e-04])}

This is useful in conjunction with tracking errors on a validation set, which you can do by passing the `X_val` and `Y_val` arguments and then inspecting the `.best_val_loss_itr` instance attribute.

In [11]:
ngb = NGBRegressor()
ngb.fit(X_reg_train, Y_reg_train, X_val=X_reg_test, Y_val=Y_reg_test) # use a validation set instead of test set here in your own work
print(ngb.best_val_loss_itr)
best_preds = ngb.predict(X_reg_test, max_iter=ngb.best_val_loss_itr)

[iter 0] loss=3.6639 val_loss=3.5325 scale=0.5000 norm=3.4373
[iter 100] loss=3.1102 val_loss=3.1139 scale=1.0000 norm=3.9032
[iter 200] loss=2.4635 val_loss=2.6144 scale=2.0000 norm=4.0454
[iter 300] loss=2.0046 val_loss=2.6142 scale=2.0000 norm=3.1239
[iter 400] loss=1.8151 val_loss=2.8736 scale=1.0000 norm=1.4106
253


## Early Stopping

NGBoost also has early stopping. If an integer `early_stopping_rounds` and a validation set (`X_val`,`Y_val`) are passed to `fit()`, the algorithm will stop running after the validation loss has increased for `early_stopping_rounds` of consecutive iterations. 

In [14]:
_ = NGBRegressor().fit(X_reg_train, Y_reg_train, X_val=X_reg_test, Y_val=Y_reg_test, early_stopping_rounds=2)

[iter 0] loss=3.6639 val_loss=3.5325 scale=0.5000 norm=3.4373
[iter 100] loss=3.1102 val_loss=3.1142 scale=1.0000 norm=3.9032
[iter 200] loss=2.4635 val_loss=2.6150 scale=2.0000 norm=4.0454
== Early stopping achieved.
== Best iteration / VAL 245 (val_loss=2.5231)


## Using `sklearn` Model Selection

`sklearn` methods are compatible with NGBoost.

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

b1 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=2)
b2 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=4)

param_grid = {
    'n_estimators': [20, 50],
    'minibatch_frac': [1.0, 0.5],
    'Base': [b1, b2]
}

ngb = NGBRegressor(Dist=Normal, verbose=False)

grid_search = GridSearchCV(ngb, param_grid=param_grid, cv=5)
grid_search.fit(X_reg_train, Y_reg_train)
print(grid_search.best_params_)

{'Base': DecisionTreeRegressor(criterion='friedman_mse', max_depth=4, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best'), 'minibatch_frac': 0.5, 'n_estimators': 20}
